In [1]:
import os
import zipfile
import math

import numpy as np
import pyUSID as pu
import pycroscopy as px
import h5py
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

/Users/wesleytatum/miniconda3/lib/python3.6/site-packages/pycroscopy/__init__.py:31: FutureWarning: Contents of pycroscopy.core such as hdf_utils, plot_utils have been moved to pyUSID but will continue to be available implicitly till the next release. Please update import statements to import such modules directly frompyUSID. See - https://pycroscopy.github.io/pycroscopy/whats_new.html under June 28 2018
  'pyUSID. See - https://pycroscopy.github.io/pycroscopy/whats_new.html under June 28 2018', FutureWarning)


In [22]:
def igor_to_hdf5(filepath):
    # 
    # returns an HDF5 format file
    
    translator = px.io.IgorIBWTranslator()
    h5_file = translator.translate(filepath)
    
    return h5_file

In [24]:
def bruker_to_hdf5(filepath):
    # 
    # returns an HDF5 format file
    
    translator = px.io.BrukerAFMTranslator()
    h5_file = translator.translate(filepath)
    
    return h5_file

In [27]:
def mk_USID_file(path):
    
    h5_fl = mkh5file(path)
    
    used_file = pu.USIDataset(h5_fl)
    
    return usid_file

In [28]:
def mk_h5_file(path):

    h5_fl = h5py.File(path, mode = 'r')
    
    return h5_fl

In [2]:
from scipy import signal

def sobel(x):
    fr = np.array([[0, 1, 0, -1, 0], [1, 2, 0, -2, -1], [2, 4, 0, -4, 2], [ 1,  2,  0, -2,  1], [0,  1,  0, -1, 0]])
    fc = np.array([[0, 1, 2,  1, 0], [1, 2, 4,  2,  1], [0, 0, 0,  0, 0], [-1, -2, -4, -2, -1], [0, -1, -2, -1, 0]])
    yr = signal.convolve2d(x, fr, boundary="symm", mode="same")
    yc = signal.convolve2d(x, fc, boundary="symm", mode="same")
    return np.sqrt(yr**2 + yc**2)
    

In [3]:
def show(X):
    fig = pyplot.figure(figsize=(20, 40), dpi= 80, facecolor='w', edgecolor='k')
    cnt = 1
    for i in range(6):
        x = X[:,:,i]
        pyplot.subplot(6,2,cnt)
        pyplot.title(fields[i])
        pyplot.imshow(x)
        pyplot.colorbar()
        
        cnt += 1
        
        pyplot.subplot(6,2,cnt)
        pyplot.title(f"{fields[i]} Gradient")
        #pyplot.imshow(np.abs(np.gradient(x**2)[0]))
        pyplot.imshow(sobel(x))
        pyplot.colorbar()
        
        cnt += 1
    pyplot.tight_layout()

In [23]:
from scipy import signal

def extract_outliers_pos(X, height_index = 4, threshold=4):
    x = X[:,:,height_index]
    flt =    np.array([[0.5, 0.5, 0.5],
                       [0.5, 1, 0.5],
                       [0.5, 0.5, 0.5]])
    y = signal.convolve2d(x, flt, boundary='symm', mode='same')
    
    y[y<0] = 0
    
    u = np.mean(y)
    s = np.std(y)
    z = np.abs((u-y)/s)
    return z > threshold

def extract_outliers_neg(X, height_index = 4, threshold=4):
    x = X[:,:,height_index]
    flt =    np.array([[0.5, 0.5, 0.5],
                       [0.5, 1, 0.5],
                       [0.5, 0.5, 0.5]])
    
    y = signal.convolve2d(x, flt, boundary='symm', mode='same')
    
    y[y>0] = 0
    
    u = np.mean(y)
    s = np.std(y)
    z = np.abs((u-y)/s)
    return z > threshold

def extract_outliers(X, height_index = 4, threshold = 4):
    pos_outliers = extract_outliers_pos(X, height_index, threshold)
    neg_outliers = extract_outliers_neg(X, height_index, threshold)
    outliers = pos_outliers + neg_outliers
    return outliers

# X = get_data(75, 10, 4, 2)
# fig = plt.figure(figsize=(20,10))

# plt.subplot(121)
# plt.imshow(X[:,:,4])
# plt.subplot(122)
# plt.imshow(extract_outliers(X))

In [8]:
from collections import Counter
from sklearn.mixture import GaussianMixture

def segment(X, outliers, normal=True):
    n, m, b = X.shape
    
    if normal:
        maxi_vector = [np.max(np.abs(X[:,:,i])) for i in range(b)]
        normal_X = np.zeros(X.shape)
        for i in range(b):
            normal_X[:,:,i] = X[:,:,i] / maxi_vector[i]
    else:
        normal_X = X
    
    n, m, b = normal_X.shape
    V = [normal_X[i,j,:] for i in range(m) for j in range(n) if not outliers[i,j]]
    V_full = [normal_X[i,j,:] for i in range(n) for j in range(m)]
        
    gmm = GaussianMixture(n_components=3, covariance_type='full')
    gmm.fit(V)
    
    l = gmm.predict(V_full)
    return l.reshape(n,m)

In [35]:
qnm_h5_fl = os.path.abspath('/Users/wesleytatum/Dropbox/Wes nanowires/QNM/2018-07-31_QNM_HDF5/2018-07-31_WT2-97_QNM_1_4um_000.h5')
# cafm_h5_fl = os.path('/Users/wesleytatum/Dropbox/Wes nanowires/cAFM/20180731_cAFM_HDF5/WT2-97_cAFM0005.h5')
# amfm_h5_fl = os.path('/Users/wesleytatum/Dropbox/Wes nanowires/AMFM/20180731_AMFM_HDF5/WT2-97_AMFM_0003.h5')

In [36]:
h5_fl = mk_h5_file(qnm_h5_fl)

pu.io.hdf_utils.print_tree(h5_fl)

OSError: Unable to open file (unable to open file: name = '/Users/wesleytatum/Dropbox/Wes nanowires/QNM/2018-07-31_QNM_HDF5/2018-07-31_WT2-97_QNM_1_4um_000.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [9]:


# X_original = get_data(63)
# X_original = np.load("/Users/ponl/Dropbox (Blacksky)/MATDAT2018/AFM/2ComponentFilms/Combined/PTB7PC71BM_CBonly_ascast_fresh_500 nm_combined.npy")
X_original[:,:,0] = np.arrapy(qnm_height[]

O = extract_outliers(X_original)
X_original[O,:] = 0

# NOTE for old data we want to get rid of height
#X = np.delete(X_original, 4, axis=2)

scaled_x = X_original.copy()
scaled_x[O, :] = 0

L = segment(scaled_x, O, False)
L2 = segment(scaled_x, O, True)

L += 1 # move from labels 0,1 to 1,2
L *= (1-O) # outliers map to label 0

L2 += 1 # move from labels 0,1 to 1,2
L2 *= (1-O) # outliers map to label 0

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ponl/Dropbox (Blacksky)/MATDAT2018/AFM/2ComponentFilms/Combined/PTB7PC71BM_CBonly_ascast_fresh_500 nm_combined.npy'

In [10]:
pyplot.figure(figsize=(20,40))

cnt = 1
for i in range(6):
    pyplot.subplot(6,2,cnt)
    pyplot.title(fields[i])
    pyplot.imshow(scaled_x[:,:,i])
    cnt += 1
    
    pyplot.subplot(6,2,cnt)
    pyplot.title("Segmentation")
    pyplot.imshow(L2)
    cnt += 1
    
pyplot.tight_layout()

NameError: name 'pyplot' is not defined

In [11]:
for i in range(6):
    pyplot.figure(figsize=(20,10))
    
    pyplot.subplot(1,2,1)
    pyplot.title(fields[i])
    pyplot.imshow(scaled_x[:,:,i])
    
    pyplot.subplot(1,2,2)
    pyplot.hist(X_original[:,:,i][L2 == 1], 100, fc=(0, 0, 1, 0.3), density=True)
    pyplot.axvline(np.mean(X_original[:,:,i][L2==1]), color='black')

    pyplot.hist(X_original[:,:,i][L2 == 2], 100, fc=(1, 0, 0, 0.3), density=True)
    pyplot.axvline(np.mean(X_original[:,:,i][L2==2]), color='black')

    pyplot.hist(X_original[:,:,i][L2 == 3], 100, fc=(0, 1, 0, 0.3), density=True)
    pyplot.axvline(np.mean(X_original[:,:,i][L2==3]), color='black')
    
    pyplot.grid()
    pyplot.show()

NameError: name 'pyplot' is not defined

In [12]:
from scipy.spatial import KDTree

def compute_distance(B):
    def onboundary(x):
        if not B[x]:
            return False
        N = get_neighbors(x, B)
        for n in N:
            if not B[n]:
                return True
        return False

    n = B.shape
    print("Computing Positives")
    positives = np.array([(i,j) for i in range(n[0]) for j in range(n[1]) if B[i,j] and onboundary((i,j))])
    print("Computing KD Tree")
    kd = KDTree(positives)
    D = np.empty(B.shape, float)

    print("Computing Distances")
    for i in range(n[0]):
        for j in range(n[1]):
                d, I = nn = kd.query((i,j))
                if not B[i,j]:
                    d *= -1
                D[i,j] = d
    return D

def get_neighbors(xc, X):
    N = []
    for j in range(-1, 2):
        for k in range(-1, 2):
            if j == 0 and k == 0:
                continue
            nr = xc[0] + j
            nc = xc[1] + k
            if nr < 0 or nc < 0 or nr >= X.shape[0] or nc >= X.shape[1]:
                continue
            N.append((xc[0]+j, xc[1]+k))
    return N

In [13]:
import numpy as np
from collections import defaultdict

class PersistenceWatershed(object):
    def __init__(self, arr):
        self.WSHED = 0
        self.INIT = -1
        self.arr = arr
        self.dim = self.arr.shape
        self.lab = np.empty(self.dim, dtype=int)
        self.lab.fill(self.INIT)
        self.dual = {}
        self.current_label = 1
        self.maxes = {}
        self.mt = None

    def train(self):
        # Mean center
        u = np.mean(self.arr.flatten())
        s = np.std(self.arr.flatten())
        self.arr -= u
        self.arr /= s
        
        rank = np.argsort(self.arr.flatten())
        for r in rank[::-1]:
            p = np.unravel_index(r, self.dim)
            L = self.get_neighbor_labels(p)
            
            if len(L) == 0:
                # assign new label
                l = self.current_label
                self.current_label += 1
                
                self.lab[p] = l
                self.dual[l] = {}
                self.maxes[l] = self.arr[p]
                
            elif len(L) == 1:
                # assign existing label
                self.lab[p] = list(L)[0]
                
            else:
                # Watershed - Connect adjacent labels
                w = self.arr[p]
                self.lab[p] = self.WSHED
                
                for l0 in L:
                    for l1 in L:
                        if l0 == l1:
                            continue
                        
                        n0, n1 = [min(l0,l1), max(l0,l1)]
                        if n1 not in self.dual[n0]:
                            self.dual[n0][n1] = -1 * np.inf
                            
                        self.dual[n0][n1] = max(self.dual[n0][n1], w)
                        
        # Create edge list
        edges = []
        for n0 in self.dual:
            for n1 in self.dual[n0]:
                edges.append((n0, n1, self.dual[n0][n1]))
        
        self.mt = PersistenceWatershed.merge_tree(self.maxes, edges)

    def apply_threshold(self, t):
        # Build relabel dictionary
        relabel = {i: i for i in range(self.current_label)}
        for e in self.mt:
            if e[2] < t:
                relabel[e[0]] = e[1]
        
        # Perform relabeling
        rlab = np.empty(self.dim, dtype=int)
        wvox = []
        for i in range(self.lab.size):
            p = np.unravel_index(i, self.dim)
            l = self.lab[p]
            if l == self.WSHED:
                rlab[p] = 0
                wvox.append(p)
                continue
            rlab[p] = PersistenceWatershed.find(relabel, self.lab[p])

        # Clean up false watersheds
        for p in wvox:
            L = self.get_neighbor_labels(p, rlab)
            if len(L) == 1:
                rlab[p] = list(L)[0]
                
        # Clean labels to avoid skips
        ol2cl = {k: i+1 for i, k in enumerate(set(rlab.flatten())) if k != 0}
        ol2cl[0] = 0
        for i in range(rlab.shape[0]):
            for j in range(rlab.shape[1]):
                rlab[i,j] = ol2cl[rlab[i,j]]
                
        return rlab

    def get_neighbor_labels(self, p, source=None):
        N = PersistenceWatershed.get_neighbors(p, self.arr.shape)
        L = set()
        for n in N:
            l = source[n] if source is not None else self.lab[n]
            if l != self.INIT and l != self.WSHED:
                L.add(l)
        return L
    
    @staticmethod
    def get_neighbors(p, bounds, k=1):
        N = [(p[0]+i, p[1]+j) for i in range(-k, k+1) for j in range(-k, k+1)]
        N = [n for n in N if (PersistenceWatershed.inbounds(n, bounds) and not (n[0] == p[0] and n[1] == p[1]))]
        return N

    @staticmethod
    def inbounds(p, bounds):
        return np.prod([0 <= p[i] < bounds[i] for i in range(2)])
    
    @staticmethod
    def find(components, u):
        while components[u] != u:
            u = components[u]
        return u

    @staticmethod
    def merge_tree(maxima, edges):
        pairs = []
        values = {}
        components = {}

        for (u,val) in maxima.items():
            values[u] = val
            components[u] = u

        # Sort by edge height
        edges.sort(key=lambda x: x[2], reverse=True)

        for e in edges:
            u, v, val = e
            uc = PersistenceWatershed.find(components, u)
            vc = PersistenceWatershed.find(components, v)
            if uc == vc:
                continue

            if values[vc] < values[uc]:
                uc, vc = vc, uc

            # source, target, persistence
            pairs.append((uc, vc, values[uc] - val))
            components[uc] = components[vc]

        return pairs

In [14]:
H = X[:,:,4]
H = (H - np.mean(H)) / np.std(H)

flt = np.ones((3,3))
flt[1,1] = 2
flt /= 2
H = convolve2d(H, flt, mode="same")

pws = PersistenceWatershed(H)
pws.train()

NameError: name 'X' is not defined

In [18]:
H = X[:,:,4]
H = (H - np.mean(H)) / np.std(H)

flt = np.ones((3,3))
flt[1,1] = 2
flt /= 2
H = convolve2d(H, flt, mode="same")

pws = PersistenceWatershed(H)
pws.train()

NameError: name 'X' is not defined

In [19]:
L = pws.apply_threshold(1)

NameError: name 'pws' is not defined

In [20]:
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

def embed(X, ndim=3, outliers=None):
    n, m, d = X.shape
    
    if outliers is None:
        outliers = set()
    else:
        oset = set()
        for i in range(n):
            for j in range(m):
                if outliers[i,j]:
                    oset.add((i,j))
        outliers = oset

    # -- Build feature embedding
    V = []
    V_full = []
    for i in range(n):
        for j in range(m):
            V_full.append(X[i, j, :])
            
            if (i,j) in outliers:
                continue
                
            V.append(X[i, j, :])
    
    # Scale
    sst = StandardScaler()
    V_scaled = sst.fit_transform(np.array(V))
    V_full_scaled = sst.transform(np.array(V_full))
    
    # Covariance Matrix
    pca = PCA(n_components=ndim)
    
    pca.fit(V_scaled)
    Y = pca.transform(V_full_scaled)
    
    return Y

def reduce_and_fit_gmm(X, ndim=3, ncmp=2):
    O = extract_outliers(X)
    Y = embed(X, ndim=ndim, outliers=O)
    gmm = GaussianMixture(n_components=ncmp)
    gmm.fit(Y)
    L = np.reshape(gmm.predict(Y), (X.shape[0], X.shape[1]))
    L += 1
    L *= (1-O)
    return L

# X = get_data(7)
L = reduce_and_fit_gmm(X, ndim=3, ncmp=2)

pyplot.figure(figsize=(15,15))
pyplot.subplot(121)
pyplot.title(fields[2])
pyplot.imshow(X[:,:,2])
pyplot.subplot(122)
pyplot.imshow(L)

NameError: name 'get_data' is not defined

In [21]:
from sklearn.decomposition import PCA
from scipy.signal import convolve2d

class FeatureNeighborhoodEmbedding(object):
    def __init__(self, X, ncomp, padding=3, ndim=3, final_dim=6):
        self.X = X
        self.Z = None
        self.nrow = X.shape[0]
        self.ncol = X.shape[1]
        self.nfeatures = X.shape[2]
        
        self.padding = padding
        self.ndim = ndim
        self.ncomp = ncomp
        self.final_dim = final_dim

        self.maxes = np.zeros(self.nfeatures)
        self.mins = np.zeros(self.nfeatures)
        self.means = np.zeros(self.nfeatures)
        self.pcas = [None for _ in range(self.nfeatures)]
        self.final_pca = None
        self.gmm = GaussianMixture(n_components=ncomp, covariance_type="full")
        
    def get_window(self, i, j, d):
        Y = self.X[:,:,d]
        w = np.zeros((self.padding*2 + 1, self.padding*2 + 1))
        r = 0
        for di in range(-self.padding, self.padding+1):
            c = 0
            for dj in range(-self.padding, self.padding+1):
                u = i + di
                v = j + dj
                if 0 <= u < self.nrow and 0 <= v < self.ncol:
                    w[r,c] = Y[u,v]
                else:
                    w[r,c] = Y[i,j]
                c += 1
            r += 1
        return w
    
    def fit(self):
        print("Learing Embeddings")
        self._learn_embeddings()
        
        print("Applying Embeddings")
        self._apply_embedding()
        
        print("Learning Final Embedding")
        self._learn_final_embedding()
        
        print("Learning Mixture Model")
        self._learn_mixture_model()

    def _learn_single_embedding(self, d):
        W = []
        for i in range(self.nrow):
            for j in range(self.ncol):
                w = self.get_window(i, j, d)
                W.append(w.flatten())
                
        W = np.array(W)

        self.mins[d] = np.min(W)
        self.maxes[d] = np.max(W)
        self.means[d] = np.mean(W)
        
        W = (W - self.mins[d]) / (self.maxes[d] - self.mins[d])
        pca = PCA(n_components=self.ndim)
        pca.fit(W)
        self.pcas[d] = pca
        return

    def _learn_embeddings(self):
        # Train feature extractors
        pcas, params = [], []
        for i in range(self.nfeatures):
            print(f"\tTraining feature {i}")
            self._learn_single_embedding(i)

    def _apply_embedding(self):
        Z = np.zeros((self.nrow, self.ncol, self.ndim * self.nfeatures))

        for d in range(self.nfeatures):
            Y = (self.X[:,:,d] - self.mins[d]) / (self.maxes[d] - self.mins[d])

            pca = self.pcas[d]

            for b, cmp in enumerate(pca.components_):
                flt_size = int(np.sqrt(len(cmp)))
                flt = cmp.reshape((flt_size, flt_size))
                Z[:,:,3 * d + b] = convolve2d(Y, flt, mode="same", fillvalue=self.means[d])
                
        self.Z = Z
        
    def _learn_final_embedding(self):
        self.final_pca = PCA(n_components=self.final_dim)
        T = [self.Z[i,j,:] for i in range(self.nrow) for j in range(self.ncol)]
        e = self.final_pca.fit_transform(T)
        self.Z = e.reshape((self.nrow, self.ncol, self.final_dim))
    
    def _learn_mixture_model(self):
        T = [self.Z[i,j,:] for i in range(self.nrow) for j in range(self.ncol)]
        self.gmm.fit(T)

    def transform(self):
        T = [self.Z[i,j,:] for i in range(self.nrow) for j in range(self.ncol)]
        L = self.gmm.predict(T)
        L = L.reshape((self.Z.shape[0], self.Z.shape[1]))
        return L
    
    def fit_transform(self):
        self.fit()
        return self.transform()

In [ ]:
X_full = get_data(5)
X = get_heightless_data(5)
FNE = FeatureNeighborhoodEmbedding(X, 3, padding=3, ndim=6, final_dim=3)

FNE.fit()
L = FNE.transform()
O = extract_outliers(X_full)

In [ ]:
pyplot.figure(figsize=(20,40))

cnt = 1
for i in range(5): 
    pyplot.subplot(5,2,cnt)
    pyplot.title(fields_compact[i])
    pyplot.imshow(X[:,:,i])
    cnt += 1
    
    pyplot.subplot(5,2,cnt)
    pyplot.imshow((L + 1) * (1 - O), cmap="brg")
    cnt += 1
    
pyplot.tight_layout()